In [ ]:
import os, cv2, csv
import numpy as np
import pandas as pd
import torch

In [ ]:
def build_labels(root_dir):
    """
    Function to build labels as accident and normal for the videoes in the dataset
    """
    labels_dict = {} 
    for label, label_val in {"accident":1,"normal":0}.items():
        folder = os.path.join(root_dir, label)
        for fname in os.listdir(folder):
            key = f"{label}/{fname}"
            labels_dict[key] = label_val
    return labels_dict

In [ ]:
def build_dataset(dataset_name, video_dir, labels_dict, out_csv):
    """
    video_dir: path to the folder containing all the videos
    labels_dict: dict {filename : label} where label is 0/1
    this function is used to add metadata to data.csv
    """
    rows = []
    for fname, label in labels_dict.items():
        path = os.path.join(video_dir, fname)
        cap = cv2.VideoCapture(path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        cap.release()
        rows.append([f'{dataset_name}_{os.path.splitext(fname)[0]}', dataset_name, path, total_frames-1, fps, label]) #the frame couting starts from 0 

    with open(out_csv, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["video_id", "dataset", "path", "clip_end_frame", "fps", "label"])
        writer.writerows(rows)

In [ ]:
def show_frames(video_path,clip_end_frame):
    """
    video_path: path to a specific video
    clip_end_frame: number of frames in the video
    this is a function to test the video sampling
    """
    cap = cv2.VideoCapture(video_path)
    frame_num = 0
    idxs = np.linspace(0, clip_end_frame-1, 30).astype(int)
    for idx in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES,idx)
        ret, frame= cap.read()
        if ret:
            cv2.imshow('Frame', frame)
            cv2.waitKey(500)
        else:
            print("Frame couldnt be read")
    cap.release()
    cv2.destroyAllWindows()

In [21]:
df = pd.read_csv("data.csv") # we have saved all the videos to data.csv
def save_frames(video_path, clip_end_frame):
    """
    video_path: path to a specific video
    clip_end_frame: number of frames in the video
    
    """
    cap = cv2.VideoCapture(video_path)
    fname = os.path.splitext(video_path)[0].split('/')[-1]
    folder_name = os.path.basename(os.path.dirname(video_path))
    os.makedirs(f"C:/Users/yashr/Downloads/sampled_frames/{folder_name}-{fname}")
    idxs = np.linspace(0, clip_end_frame-1, 60).astype(int)
    for idx in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES,idx)
        ret, frame= cap.read()
        if ret:
            frame_filename = os.path.join(f"C:/Users/yashr/Downloads/sampled_frames/{folder_name}-{fname}", f"frame_{idx:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
        else:
            print("Frame couldnt be read")

In [23]:
for _, row in df.iloc[372:].iterrows():
    video_path = row["path"]
    clip_end_frame = row["clip_end_frame"]
    save_frames(video_path, clip_end_frame)

Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read
Frame couldnt be read


In [39]:
ROOT = "sampled_frames"

for folder in sorted(os.listdir(ROOT)):
    clip_path = os.path.join(ROOT, folder)
    if not os.path.isdir(clip_path):
        continue

    # collect frame files
    frames = [f for f in os.listdir(clip_path)]
    frames.sort()  # ensuring chronological order

    n = len(frames)
    if(n<60):
        last_frame_path = os.path.join(clip_path, frames[-1])
        last_img = cv2.imread(last_frame_path)
        new_name = f"frame_{i+1:04d}.jpg"  # consistent naming
        new_path = os.path.join(clip_path, new_name)
        cv2.imwrite(new_path, last_img)
        print(f"Padded {folder}: added {60 - n} frames")